## Project 1.2 - Test Functions

Group: Emmy Noether

Students: Janik Rausch (628334), Camilo Tello Breuer (633291), Ida Wöstheinreich (628428)

In [13]:
using LinearAlgebra,Random,Printf
include("Ground_state.jl")

ground_state (generic function with 1 method)

**Tests of the Hamiltonian**

The Hamiltonian must be linear, hermitian and positive-definite. Also, plane waves must be eigenfunctions of the kinetic term, with eigenvalues $$E_\textbf{k}=-\frac{1}{\mu\epsilon^2}\sum_i\left(\cos\left(\frac{2\pi}{N}k_i\right)-1\right).$$

In [3]:
function test_linearity(N::Int, params::Array{Tuple{Real, Real}})

    function test_engine(shape::Tuple{Vararg{Int}})
        ψ, ϕ = rand(ComplexF64, shape), rand(ComplexF64, shape)
        α, β = rand(ComplexF64), rand(ComplexF64)
        n = lattice(N, length(shape))

        res = fill(1.0, (3,))

        for i in 1:3
            μ, ϵ = params[i]
            V = potential(μ, ϵ, n)
            δ = Hamiltonian(μ, ϵ, (@. α*ψ + β*ϕ), V) - α*Hamiltonian(μ, ϵ, ψ, V) - β*Hamiltonian(μ, ϵ, ϕ, V)
            res[i] = max(abs.(δ)...)
        end
        
        @printf "residues = [%.3e, %.3e, %.3e]" res...
    end
    
    println("Linearity:")
    print("D=1: "); test_engine((N,))
    print("\nD=2: "); test_engine((N,N))
    print("\nD=3: "); test_engine((N,N,N)); print("\n")
end

function test_hermiticity(N::Int, params::Array{Tuple{Real, Real}})

    function test_engine(shape::Tuple{Vararg{Int}})
        ψ, ϕ = rand(ComplexF64, shape), rand(ComplexF64, shape)
        α, β = rand(ComplexF64), rand(ComplexF64)
        n = lattice(N, length(shape))

        res = fill(1.0, (3,))
        
        for i in 1:3
            μ, ϵ = params[i]
            V = potential(μ, ϵ, n)
            δ = dot(ψ, Hamiltonian(μ, ϵ, ϕ, V)) - dot(Hamiltonian(μ, ϵ, ψ, V), ϕ)
            res[i] = max(abs.(δ)...)
        end
        
        @printf "residues = [%.3e, %.3e, %.3e]" res...
    end
    
    println("Hermiticity:")
    print("D=1: "); test_engine((N,))
    print("\nD=2: "); test_engine((N,N))
    print("\nD=3: "); test_engine((N,N,N)); print("\n")
end

function test_positivity(N::Int, params::Array{Tuple{Real, Real}})

    function test_engine(shape::Tuple{Vararg{Int}})
        ψ = rand(ComplexF64, shape)
        
        res = fill(1.0+im, (3,))
        
        for i in 1:3
            μ, ϵ = params[i]
            δ = dot(ψ, kinetic(μ, ϵ, ψ))
            res[i] = δ
        end
        
        print([real(res)[i] >= 0 for i in 1:3])
        @printf ", imaginary parts = [%.3e, %.3e, %.3e]" imag(res)...
    end
    
    println("Positivity:")
    print("D=1: "); test_engine((N,))
    print("\nD=2: "); test_engine((N,N))
    print("\nD=3: "); test_engine((N,N,N)); print("\n")
end

function test_eigenfunctions(N::Int, params::Array{Tuple{Real, Real}})

    function test_engine(shape::Tuple{Vararg{Int}})
        k = rand(Int16, (length(shape)))
        K = fill(k, shape)
        n = lattice(N, length(shape))
        ψ_k = @. exp(2*pi*im/N * dot(n, K))

        res = fill(1.0, (3,))
        
        for i in 1:3
            μ, ϵ = params[i]
            δ = kinetic(μ, ϵ, ψ_k) + 1/(μ*ϵ^2) * (sum(cos.(2*pi/N * k)) - length(shape)) * ψ_k
            res[i] = max(abs.(δ)...)
        end
        
        @printf "residues = [%.3e, %.3e, %.3e]" res...
    end
    
    println("Eigenfunctions:")
    print("D=1: "); test_engine((N,))
    print("\nD=2: "); test_engine((N,N))
    print("\nD=3: "); test_engine((N,N,N)); print("\n")
end

test_eigenfunctions (generic function with 1 method)

In [6]:
params = [(1,1), (20,0.1), (0.1,20)]

println("(μ, ϵ) = ", string(params)[18:end], "\n")
test_linearity(10, params)
test_hermiticity(10, params)
test_positivity(10, params)
test_eigenfunctions(10, params)

(μ, ϵ) = [(1, 1), (20, 0.1), (0.1, 20)]

Linearity:
D=1: residues = [7.105e-15, 9.155e-16, 1.164e-10]
D=2: residues = [6.355e-14, 5.329e-15, 1.041e-09]
D=3: residues = [1.705e-13, 1.037e-14, 2.945e-09]
Hermiticity:
D=1: residues = [0.000e+00, 8.882e-16, 1.164e-10]
D=2: residues = [5.684e-13, 1.465e-14, 0.000e+00]
D=3: residues = [3.638e-12, 2.274e-13, 1.333e-07]
Positivity:
D=1: Bool[1, 1, 1], imaginary parts = [2.776e-17, -2.220e-16, 1.735e-18]
D=2: Bool[1, 1, 1], imaginary parts = [-1.554e-15, -8.882e-16, -4.337e-17]
D=3: Bool[1, 1, 1], imaginary parts = [1.243e-14, -4.263e-14, -4.441e-16]
Eigenfunctions:
D=1: residues = [3.830e-13, 1.914e-12, 9.572e-15]
D=2: 

residues = [1.405e-11, 7.026e-11, 3.513e-13]
D=3: residues = [1.811e-11, 9.053e-11, 4.526e-13]


The residues are consistently of order $10^{-6}$ or smaller for all tests. They tend to increase with the number of dimensions.

**Tests of the eigenvalue algorithms**

In [9]:
function test_power_method(N::Int, tol::Float64, maxiters::Int)
    
    function test_engine(shape::Tuple{Vararg{Int}})
        M = prod(shape)
        A = randn(ComplexF64, (M,M))
        A = A' * A; A = (A + A')/2
        
        function apply_A(v::Array)
            @assert size(v) == shape
            return reshape(A*reshape(v, (M,)), shape)
        end
        
        λ, v, niters = power_method(shape, apply_A, tol, maxiters)
        δ = apply_A(v) - λ*v
        res = max(abs.(δ)...)

        @printf "residue = %.3e" res
        print(", iterations = ", niters, ", Test passes: ", res <= tol)
    end
    
    println("Power method:")
    print("D=1: "); @time "; runtime" test_engine((N,))
    print("D=2: "); @time "; runtime" test_engine((N,N))
    print("D=3: "); @time "; runtime" test_engine((N,N,N)); print("\n")
end

function test_conjugate_gradient(N::Int, tol::Float64, maxiters::Int)
    
    function test_engine(shape::Tuple{Vararg{Int}})
        M = prod(shape)
        A = randn(ComplexF64, (M,M))
        A = A' * A; A = (A + A')/2 #make A positive definite & hermitian
        b = randn(ComplexF64, shape)
        
        function apply_A(v::Array)
            @assert size(v) == shape
            return reshape(A*reshape(v, (M,)), shape)
        end
        
        x, niters = conjugate_gradient(apply_A, b, tol, maxiters)
        δ = apply_A(x) - b
        res = norm(δ)

        @printf "residue = %.3e" res
        print(", iterations = ", niters, ", test passes: ", res <= tol*norm(b))
    end

    println("Conjugate Gradient:")
    print("D=1: "); @time "; runtime" test_engine((N,))
    print("D=2: "); @time "; runtime" test_engine((N,N))
    print("D=3: "); @time "; runtime" test_engine((N,N,N)); print("\n")
end

test_conjugate_gradient (generic function with 1 method)

In [12]:
test_power_method(8, 1e-10, 10000)
test_conjugate_gradient(8, 1e-10, 10000)

Power method:
D=1: residue = 8.790e-11, iterations = 37, Test passes: true; runtime: 0.000367 seconds (1.14 k allocations: 55.961 KiB)
D=2: residue = 9.891e-11, iterations = 249, Test passes: true; runtime: 0.010445 seconds (36.20 k allocations: 2.117 MiB)
D=3: 

residue = 9.856e-11, iterations = 1291, Test passes: true; runtime: 0.750859 seconds (1.35 M allocations: 98.389 MiB, 2.72% gc time)

Conjugate Gradient:
D=1: residue = 6.149e-12, iterations = 8, test passes: true; runtime: 0.015468 seconds (231 allocations: 21.289 KiB)
D=2: residue = 5.711e-10, iterations = 98, test passes: true; runtime: 0.009507 seconds (2.75 k allocations: 1.045 MiB)
D=3: 

residue = 1.927e-09, iterations = 959, test passes: true; runtime: 0.658676 seconds (26.04 k allocations: 70.033 MiB, 2.61% gc time)



The algorithms converge quickly for small $N$, but the number of iterations and the runtime increase very quickly with $N$, especially for $D=3$.